# Preparation

## Import libraries

In [1]:
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import re
import pandas as pd
import time
import pytz
from datetime import datetime
import urllib.request, json, os, os.path, csv
from pprint import pprint

## Define function

In [2]:
def zonasi(row):
    if int(row['total_positif']) > 0:
        return('Merah')
    elif int(row['total_pdp']) > 0:
        return('Oranye')
    elif int(row['total_odp']) > 0:
        return('Hijau')
    else: return('Biru')

# Web Scrapping

In [3]:
tz = pytz.timezone('Asia/Jakarta')
default_date_update = datetime.now(tz).strftime("%d-%b-%Y")

format_header = ['scrape_date', 'date_update', 'waktu', 'provinsi', 'kabkot', 'kecamatan', 
                 'kelurahan', 'alamat', 'total_odp', 'total_pdp', 'total_positif', 'total_otg',
                 'odr_total', 'total_pp', 'total_ppdt', 'source_link', 'zona', 'Ket']

singkawang_dataset = []

now = datetime.now(tz)

url = 'https://statistik.singkawangkota.go.id/api/v1/'
now = datetime.now(tz)

with urllib.request.urlopen(url) as request:
    raw_data = json.loads(request.read().decode())
    new_corpus = {'scrape_date':now.strftime("%d-%b-%Y"),
                  'date_update':now.strftime("%d-%b-%Y"),
                  'waktu':now.strftime("%H:%M"),
                  'provinsi':'Kalimantan Barat',
                  'kabkot':'Kota Singkawang',
                  'total_odp':raw_data['message'][0]['ODP'],
                  'total_pdp':int(raw_data['message'][0]['negatif'])+int(raw_data['message'][0]['PDP']),
                  'total_positif':int(raw_data['message'][0]['positif'])+int(raw_data['message'][0]['sembuh'])+int(raw_data['message'][0]['meninggal']),
                  'source_link':url}
if new_corpus not in singkawang_dataset: singkawang_dataset.append(new_corpus)
else: print('Identical corpus already exist in dataset.')

print('Length of dataset:', len(singkawang_dataset)) #1

df = pd.DataFrame(columns=format_header)

for data in singkawang_dataset:
    df = df.append(data, ignore_index=True)

num_nan = ['total_odp', 'total_pdp', 'total_positif', 'total_otg', 'odr_total', 
           'total_pp', 'total_ppdt']
df[num_nan] = df[num_nan].fillna(0)
str_nan = ['kecamatan', 'kelurahan', 'alamat', 'Ket']
df[str_nan] = df[str_nan].fillna('')

df['zona'] = df.apply(lambda row: zonasi(row), axis=1)

df.to_excel('{0}_Kota_Singkawang.xlsx' .format(datetime.now(tz).strftime("%Y%m%d")), index=False)

print('Shape of df:', df.shape)
df.head()

Length of dataset: 1
Shape of df: (1, 18)


,scrape_date,date_update,waktu,provinsi,kabkot,kecamatan,kelurahan,alamat,total_odp,total_pdp,total_positif,total_otg,odr_total,total_pp,total_ppdt,source_link,zona,Ket
0,27-Jul-2020,27-Jul-2020,18:36,Kalimantan Barat,Kota Singkawang,,,,154,34,10,0,0,0,0,https://statistik.singkawangkota.go.id/api/v1/,Merah,
